## Voice Assistant Demo

In [ ]:
import json
from IPython.display import display, Audio
from huggingface_hub import InferenceClient, AsyncInferenceClient

tts_client = AsyncInferenceClient("http://localhost:7777/models/bark-tts")
llm_client = AsyncInferenceClient("http://localhost:7777/models/phi-3-mini-128k-chat")
whisper_client = AsyncInferenceClient("http://localhost:7777/models/distil-large-v3")

In [ ]:
history = []
while True:
    history.append({'role': 'user', 'content': input("> ")})

    out = await llm_client.chat_completion(history)
    history.append({'role': 'assistant', 'content': out.choices[-1].message.content})
    print(f'Out: {out.choices[-1].message.content}')
    audio = await tts_client.text_to_speech(out.choices[-1].message.content[:500])
    display(Audio(audio, rate=24000))
    audio.save('audio.flac')

In [ ]:
a = Audio(audio, rate=24000)

In [ ]:
with open('audio.flac', 'wb') as f:
    f.write(audio)

In [ ]:
Audio('audio.flac')

In [ ]:
await whisper_client.automatic_speech_recognition(audio)

In [ ]:
audio = await tts_client.text_to_speech(out.choices[-1].message.content[:500])

In [ ]:
display(Audio(audio, rate=24000))

In [ ]:
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio

In [ ]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

In [ ]:
recorder.save('foo.flac')

In [ ]:
display(Audio('foo.flac'))

In [118]:
import subprocess, time

# Initialize the camera stream with audio only
camera = CameraStream(constraints={'audio': True, 'video': False})
recorder = AudioRecorder(stream=camera)

# Start recording
display(recorder)

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [119]:
if ready.lower() in ['no', 'n']:
  pass
elif ready.lower() in ['yes', 'y']:
    # Attempt to save the recorded data
    with open('foo.webm', 'wb') as f:
        f.write(recorder.audio.value)
    command = ['ffmpeg', '-i', 'foo.webm', '-ac', '1', '-y', '-f', 'wav', 'file.wav', '-y', '-hide_banner', '-loglevel', 'panic']
    subprocess.run(command,stdout=subprocess.PIPE,stdin=subprocess.PIPE)
    time.sleep(5)
    # Perform automatic speech recognition
    res = await whisper_client.automatic_speech_recognition('file.wav')
    print(f"GOT: {res.text}")

GOT:  All right, I need to get started. Could you quickly check the traffic on my way to the city?


In [120]:
text = await llm_client.chat_completion([{'role': 'user', 'content': res.text}])

In [121]:
audio = await tts_client.text_to_speech(text.choices[-1].message.content)

In [122]:
Audio(audio, autoplay=True)